In [2]:
%load_ext autoreload
%autoreload 2
from utils import *

In [3]:
df = pd.read_csv("csv/airfoil.csv")

In [4]:
df.sample(5)

,frequency,angle of attack,chord length,velocity,thickness,scaled sound pressure level
50,315,0.0,0.3048,31.7,0.003313,122.765
1440,6300,12.3,0.1016,55.5,0.036823,110.124
1009,3150,4.8,0.0254,71.3,0.000849,133.378
247,1000,2.0,0.2286,71.3,0.002930,128.956
146,800,3.0,0.3048,31.7,0.005295,128.708


In [5]:
print("frequency : ", np.unique(df["frequency"].values))
print("angle of attack : ", np.unique(df["angle of attack"].values))
print("chord length : ", np.unique(df["chord length"].values))
print("velocity : ", np.unique(df["velocity"].values))
print("thickness : ", np.unique(df["thickness"].values))
print("scaled sound pressure level : ", np.unique(df['scaled sound pressure level'].values))

frequency :  [  200   250   315   400   500   630   800  1000  1250  1600  2000  2500
  3150  4000  5000  6300  8000 10000 12500 16000 20000]
angle of attack :  [ 0.   1.5  2.   2.7  3.   3.3  4.   4.2  4.8  5.3  5.4  6.7  7.2  7.3
  8.4  8.9  9.5  9.9 11.2 12.3 12.6 12.7 15.4 15.6 17.4 19.7 22.2]
chord length :  [0.0254 0.0508 0.1016 0.1524 0.2286 0.3048]
velocity :  [31.7 39.6 55.5 71.3]
thickness :  [0.00040068 0.00041229 0.00042846 0.00043947 0.00074048 0.00076193
 0.00079182 0.00081216 0.00084863 0.00087322 0.00090748 0.00093079
 0.00121072 0.00131983 0.00142788 0.00146332 0.00150092 0.00152689
 0.0015988  0.00172668 0.00193287 0.00202822 0.00209405 0.00214345
 0.002211   0.00229336 0.00243851 0.00245138 0.00251435 0.00253511
 0.00266337 0.0027238  0.00283081 0.00293031 0.00294804 0.00310138
 0.00313525 0.00331266 0.00336729 0.00346574 0.00372371 0.00392107
 0.00400603 0.00401199 0.00420654 0.00425727 0.0042862  0.0043284
 0.00433288 0.00449821 0.00452492 0.00461377 0.00473801 0.0

In [6]:
df['scaled sound pressure level'].describe()

count    1503.000000
mean      124.835943
std         6.898657
min       103.380000
25%       120.191000
50%       125.721000
75%       129.995500
max       140.987000
Name: scaled sound pressure level, dtype: float64

In [7]:
v_min = df['scaled sound pressure level'].min()

## 2 classes :

In [8]:
v_med = df['scaled sound pressure level'].quantile(0.5)

In [9]:
df2 = df.copy()

In [53]:
print("mean frequency :")
print(df[(df['scaled sound pressure level'] >= v_min) & (df['scaled sound pressure level'] < v_med)]['frequency'].mean())
print(df[df['scaled sound pressure level'] >= v_med]['frequency'].mean())

mean frequency :
3922.37017310253
1851.7686170212767


In [54]:
print("mean angle of attack :")
print(df[(df['scaled sound pressure level'] >= v_min) & (df['scaled sound pressure level'] < v_med)]['angle of attack'].mean())
print(df[df['scaled sound pressure level'] >= v_med]['angle of attack'].mean())

mean angle of attack :
7.654727030625832
5.911037234042554


In [55]:
print("mean chord length :")
print(df[(df['scaled sound pressure level'] >= v_min) & (df['scaled sound pressure level'] < v_med)]['chord length'].mean())
print(df[df['scaled sound pressure level'] >= v_med]['chord length'].mean())

mean chord length :
0.15118242343541946
0.1219335106382979


In [56]:
print("mean velocity :")
print(df[(df['scaled sound pressure level'] >= v_min) & (df['scaled sound pressure level'] < v_med)]['velocity'].mean())
print(df[df['scaled sound pressure level'] >= v_med]['velocity'].mean())

mean velocity :
49.15326231691079
52.56595744680851


In [57]:
print("mean thickness :")
print(df[(df['scaled sound pressure level'] >= v_min) & (df['scaled sound pressure level'] < v_med)]['thickness'].mean())
print(df[df['scaled sound pressure level'] >= v_med]['thickness'].mean())

mean thickness :
0.014285190802929431
0.007998752573138297


In [11]:
df2.loc[(df2['scaled sound pressure level'] >= df2['scaled sound pressure level'].min()) &(df2['scaled sound pressure level'] <= v_med) ,'scaled sound pressure level'] = 2
df2.loc[df2['scaled sound pressure level'] >= v_med ,'scaled sound pressure level'] = 1

In [12]:
m = 5 # input dimension
vectors = df2.values
dataset = LabeledSet(m)
dataset.addExamples(vectors[:,0:m], vectors[:,m:])

In [13]:
for i in range(2):
    print("number of instances of class " + str(i+1) + " : " + str(np.count_nonzero(dataset.y == i+1)))

number of instances of class 1 : 751
number of instances of class 2 : 752


In [61]:
NMI1(dataset)

0.009045380167317384

In [14]:
nb_folds = 4

folds = get_folds(dataset, nb_folds)

In [15]:
g = Log()
h = Sum()

# rank Shannon discrimination measure
f_r = Dsr()
rsdm = Gdm(h, g, f_r)

# conditional Shannon entropy
f = Ds()
sdm = Gdm(h, g, f)

# pessimistic rank discrimination measure
f_p = Mindsr()
g_p = Frac() 
prdm = Gdm(h, g_p, f_p)

# rank Gini discrimination measure 
g_g = One_minus()
rgdm = Gdm(h, g_g, f_r)

# Gini discrimination measure
gdm = Gdm(h, g_g, f)

# H_M
h_m = Max()
g_m = One_minus_square()
H_m = Gdm(h_m, g_m, f_r)

# H_Q
f_q = Avgdsr()
h_q = Square_root()
H_q = Gdm(h_q, g_g, f_q)

In [15]:
# Rank Shannon discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rsdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.6893492907801418
standard deviation of accuracy:  0.10809451434909907
average depth :  17.0
standard deviation of depth:  1.224744871391589
average number of leaves :  99.75
standard deviation of number of leaves:  5.0682837331783235
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8781927511377756
standard deviation of ratio:  0.012357426664691437
average number of pairs used for ratio computing :  28.5
standard deviation of number of pairs:  2.29128784747792


In [33]:
# Shannon entropy
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(sdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.6966719858156029
standard deviation of accuracy:  0.11447635391744482
average depth :  13.5
standard deviation of depth:  0.8660254037844386
average number of leaves :  78.25
standard deviation of number of leaves:  5.539629951540085
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8786074364167358
standard deviation of ratio:  0.012863256915171942
average number of pairs used for ratio computing :  24.5
standard deviation of number of pairs:  2.8722813232690143


In [34]:
# Rank Gini discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(rgdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.695354609929078
standard deviation of accuracy:  0.11515076621023802
average depth :  17.5
standard deviation of depth:  1.5
average number of leaves :  102.5
standard deviation of number of leaves:  7.794228634059948
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8835781167068247
standard deviation of ratio:  0.013604402401463414
average number of pairs used for ratio computing :  28.5
standard deviation of number of pairs:  2.598076211353316


In [35]:
# Gini entropy
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(gdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.7033280141843972
standard deviation of accuracy:  0.11940007762421337
average depth :  12.25
standard deviation of depth:  0.4330127018922193
average number of leaves :  79.25
standard deviation of number of leaves:  5.673402858955108
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.874227691394926
standard deviation of ratio:  0.01886171040678174
average number of pairs used for ratio computing :  29.0
standard deviation of number of pairs:  1.224744871391589


In [16]:
# Pessimistic rank discrimination measure
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(prdm, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.7358670212765959
standard deviation of accuracy:  0.046658956867874656
average depth :  15.5
standard deviation of depth:  2.29128784747792
average number of leaves :  91.5
standard deviation of number of leaves:  4.5
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8924351424396262
standard deviation of ratio:  0.015874771473168736
average number of pairs used for ratio computing :  24.5
standard deviation of number of pairs:  2.692582403567252


In [37]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_m, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.7325177304964539
standard deviation of accuracy:  0.051272530812067875
average depth :  19.0
standard deviation of depth:  1.5811388300841898
average number of leaves :  138.25
standard deviation of number of leaves:  9.807522622966516
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.7794260696899061
standard deviation of ratio:  0.02777937760008017
average number of pairs used for ratio computing :  33.0
standard deviation of number of pairs:  3.0


In [17]:
acc = []
leaves = []
depth = []
ratio = []
pairs = []

for i in range(nb_folds):
    test_set = folds[i]
    train_set = LabeledSet(m)
    for j in range(0, nb_folds):
        if i != j:
            train_set.addExamples(folds[j].x, folds[j].y)
    tree = RDMT(H_q, "shannon", 0, 100, 0.01 * train_set.size(), [1, 2])
    tree.train(train_set)
    acc.append(tree.accuracy(test_set))
    leaves.append(tree.get_nb_leaves())
    depth.append(tree.get_depth())
    ratio.append(tree.get_ratio_non_monotone_pairs())
    pairs.append(tree.get_total_pairs())
        
avg_accuracy = np.mean(acc)
std_accuracy = np.std(acc)
avg_leaves = np.mean(leaves)
std_leaves = np.std(leaves)
avg_depth = np.mean(depth)
std_depth = np.std(depth)
avg_ratio = np.mean(ratio)
std_ratio = np.std(ratio)
avg_pairs = np.mean(pairs)
std_pairs = np.std(pairs)

print("average accuracy : ", avg_accuracy)
print("standard deviation of accuracy: ", std_accuracy)
print("average depth : ", avg_depth)
print("standard deviation of depth: ", std_depth)
print("average number of leaves : ", avg_leaves)
print("standard deviation of number of leaves: ", std_leaves)
print("average ratio between number of pairwise non-monotone label comparisons and number of pairs : ", avg_ratio)
print("standard deviation of ratio: ", std_ratio)
print("average number of pairs used for ratio computing : ", avg_pairs)
print("standard deviation of number of pairs: ", std_pairs)

average accuracy :  0.6627109929078014
standard deviation of accuracy:  0.09661871353633952
average depth :  18.75
standard deviation of depth:  3.2691742076555053
average number of leaves :  129.25
standard deviation of number of leaves:  3.112474899497183
average ratio between number of pairwise non-monotone label comparisons and number of pairs :  0.8649076223128676
standard deviation of ratio:  0.03984951255578986
average number of pairs used for ratio computing :  36.25
standard deviation of number of pairs:  0.4330127018922193
